# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [30]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [31]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [32]:
import os
from glob import glob

# Write your code below.
price_data_path = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [33]:
# Write your code below.
import pandas as pd

dd_px = dd.read_parquet(parquet_files)

dd_lags = (
    dd_px
    .groupby('ticker', group_keys = False)
    .apply(
        lambda x: x.assign(
            Close_lag_1 = x['Close'].shift(1),
            Adj_Close_lag_1 = x['Adj Close'].shift(1)
        ),
        meta = pd.DataFrame(
            data = {
                'Date': 'datetime64[ns]',
                'Open': 'f8',
                'High': 'f8',
                'Low': 'f8',
                'Close': 'f8',
                'Adj Close': 'f8',
                'Volume': 'i8',
                'source': 'object',
                'ticker': 'object',
                'Year': 'int32',
                'Close_lag_1': 'f8',
                'Adj_Close_lag_1': 'f8'
            },
            index = pd.Index([], dtype=pd.StringDtype(), name='ticker')
        )
    )
)

dd_rets = dd_lags.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

dd_feat = dd_rets.assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [34]:
# Write your code below.
df_feat = dd_feat.compute()

df_feat_with_moving_avg = (
    df_feat
    .groupby('ticker', group_keys = False)
    .apply(
        lambda x: x.assign(
            returns_moving_average_10_days = x['returns'].rolling(10).mean()
        )
    )
)

df_feat_with_moving_avg

/var/folders/91/tzhvh9qd33v4c58ym1d5c0x80000gn/T/ipykernel_10067/2085134839.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_moving_average_10_days
4625,1994-03-04,52.500000,52.500000,52.000000,52.500000,21.373365,1200.0,ANAT.csv,ANAT,1994,51.750000,21.068031,0.014493,0.500000,0.003081
4625,2005-11-17,46.939999,47.639999,46.740002,47.639999,46.403221,709300.0,FICO.csv,FICO,2005,46.919998,45.701916,0.015345,0.899998,0.006885
4625,2011-10-28,27.660000,27.660000,27.139999,27.219999,24.249929,1286900.0,FLIR.csv,FLIR,2011,27.680000,24.659733,-0.016619,0.520000,-0.000153
4625,1998-07-02,5.250000,5.250000,5.000000,5.062500,4.865292,152300.0,HL.csv,HL,1998,5.250000,5.045489,-0.035714,0.250000,0.001396
4625,2008-07-07,21.660000,21.770000,20.910000,21.559999,21.559999,4291300.0,HOLX.csv,HOLX,2008,21.600000,21.600000,-0.001852,0.860001,-0.043128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80186,2015-12-24,340.369995,340.869995,335.179993,337.399994,301.822784,240300.0,BLK.csv,BLK,2015,339.640015,303.826630,-0.006595,5.690002,-0.001930
80187,2015-12-28,336.940002,342.820007,334.880005,342.450012,306.340302,865800.0,BLK.csv,BLK,2015,337.399994,301.822784,0.014967,7.940002,0.006075
80188,2015-12-29,344.769989,347.149994,341.989990,343.600006,307.369049,480200.0,BLK.csv,BLK,2015,342.450012,306.340302,0.003358,5.160004,0.007396
80189,2015-12-30,342.359985,345.190002,341.220001,342.049988,305.982513,356900.0,BLK.csv,BLK,2015,343.600006,307.369049,-0.004511,3.970001,0.003835


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

It was not necessary to convert to pandas since Dask also has a rolling function to calculate the moving average return. In this case Dask was not the best solution since the dataset is not larger than the available memory and the pandas syntax is simpler without needing meta specifications.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.